# Tutoriel pytorch - TP3 - IFT725

Tel que mentionné dans l'énoncé du travail, vous devez recopier les blocs de code du tutoriel suivant

https://pytorch.org/tutorials/beginner/pytorch_with_examples.html

en donnant, pour chaque bloc, une description en format "markdown" de son contenu.

# Tensors
## Warm-up: numpy

Before introducing PyTorch, we will first implement the network using numpy.

Numpy provides an n-dimensional array object, and many functions for manipulating these arrays. Numpy is a generic framework for scientific computing; it does not know anything about computation graphs, or deep learning, or gradients. However we can easily use numpy to fit a two-layer network to random data by manually implementing the forward and backward passes through the network using numpy operations:

In [ ]:
import numpy as np

# N is batch size, Din is input dimension, 
# H is hidden layer dimension and Dout is output dimension
N, Din, H, Dout = 64, 1000, 100, 10

# Create some random data.
x = np.random.randn(N, Din)
y = np.random.randn(N, Dout)

w1 = np.random.randn(Din, H)
w2 = np.random.randn(H, Dout)

lr = 1e-6
for e in range(500):
    # Forward Pass
    h = x.dot(w1)
    h_relu = np.maximum(h, 0)
    y_pred = h_relu.dot(w2)

    # Compute data
    loss = np.square(y_pred - y).sum()
    print(e, loss)

    # Backward Pass
    grad_y_pred = 2.0* (y_pred - y)
    grad_w2 = h_relu.T.dot(grad_y_pred)
    grad_h_relu = grad_y_pred.dot(w2.T)
    grad_h = grad_h_relu.copy()
    grad_h[h < 0] = 0
    grad_w1 = x.T.dot(grad_h)

    # Weight update
    w1 -= lr * grad_w1
    w2 -= lr * grad_w2

# PyTorch: Tensors

Numpy is a great framework, but it cannot utilize GPUs to accelerate its numerical computations. For modern deep neural networks, GPUs often provide speedups of 50x or greater, so unfortunately numpy won’t be enough for modern deep learning.

Here we introduce the most fundamental PyTorch concept: the Tensor. A PyTorch Tensor is conceptually identical to a numpy array: a Tensor is an n-dimensional array, and PyTorch provides many functions for operating on these Tensors. Behind the scenes, Tensors can keep track of a computational graph and gradients, but they’re also useful as a generic tool for scientific computing.

Also unlike numpy, PyTorch Tensors can utilize GPUs to accelerate their numeric computations. To run a PyTorch Tensor on GPU, you simply need to cast it to a new datatype.

Here we use PyTorch Tensors to fit a two-layer network to random data. Like the numpy example above we need to manually implement the forward and backward passes through the network:

In [ ]:
import torch

dtype = torch.float
device = torch.device("cpu")       # gpu = cuda

# N is batch size, Din is input dimension, 
# H is hidden layer dimension and Dout is output dimension
N, Din, H, Dout = 64, 1000, 100, 10

x = torch.randn(N, Din, device=device, dtype=dtype)
y = torch.randn(N, Dout, device=device, dtype=dtype)

w1 = torch.randn(Din, H, device=device, dtype=dtype)
w2 = torch.randn(H, Dout, device=device, dtype=dtype)

lr = 1e-6
for e in range(500):
    # Forward pass 
    h = x.mm(w1)
    h_relu = h.clamp(min=0)
    y_pred = h_relu.mm(w2)

    loss = (y_pred - y).pow(2).sum().item()
    if e % 100 == 0:
        print(e, loss)

    # Backward pass
    grad_y_pred = 2.0* (y_pred - y)
    grad_w2 = h_relu.t().mm(grad_y_pred)
    grad_h_relu = grad_y_pred.mm(w2.t())
    grad_h = grad_h_relu.clone()
    grad_h[h < 0] = 0
    grad_w1 = x.t().mm(grad_h)

    # Weight update
    w1 -= lr * grad_w1
    w2 -= lr * grad_w2

# Autograd
## PyTorch: Tensors and autograd

In the above examples, we had to manually implement both the forward and backward passes of our neural network. Manually implementing the backward pass is not a big deal for a small two-layer network, but can quickly get very hairy for large complex networks.

Thankfully, we can use automatic differentiation to automate the computation of backward passes in neural networks. The autograd package in PyTorch provides exactly this functionality. When using autograd, the forward pass of your network will define a computational graph; nodes in the graph will be Tensors, and edges will be functions that produce output Tensors from input Tensors. Backpropagating through this graph then allows you to easily compute gradients.

This sounds complicated, it’s pretty simple to use in practice. Each Tensor represents a node in a computational graph. If x is a Tensor that has x.requires_grad=True then x.grad is another Tensor holding the gradient of x with respect to some scalar value.

Here we use PyTorch Tensors and autograd to implement our two-layer network; now we no longer need to manually implement the backward pass through the network:

In [ ]:
import torch

dtype = torch.float
device = torch.device("cpu")       # gpu = cuda

# N is batch size, Din is input dimension, 
# H is hidden layer dimension and Dout is output dimension
N, Din, H, Dout = 64, 1000, 100, 10

x = torch.randn(N, Din, device=device, dtype=dtype)
y = torch.randn(N, Dout, device=device, dtype=dtype)

w1 = torch.randn(Din, H, device=device, dtype=dtype, requires_grad=True)
w2 = torch.randn(H, Dout, device=device, dtype=dtype, requires_grad=True)

lr = 1e-6 
for e in range(500):
    y_pred = x.mm(w1).clamp(min=0).mm(w2)

    loss = (y_pred - y).pow(2).sum()
    if e % 100 == 0:
        print(e, loss.item())
    
    loss.backward()


    with torch.no_grad():
        w1 -= lr* w1.grad
        w2 -= lr* w2.grad

        w1.grad.zero_()
        w2.grad.zero_()

## PyTorch: Defining new autograd functions

Under the hood, each primitive autograd operator is really two functions that operate on Tensors. The forward function computes output Tensors from input Tensors. The backward function receives the gradient of the output Tensors with respect to some scalar value, and computes the gradient of the input Tensors with respect to that same scalar value.

In PyTorch we can easily define our own autograd operator by defining a subclass of torch.autograd.Function and implementing the forward and backward functions. We can then use our new autograd operator by constructing an instance and calling it like a function, passing Tensors containing input data.

In this example we define our own custom autograd function for performing the ReLU nonlinearity, and use it to implement our two-layer network:

In [ ]:
import torch 

class MyReLU(torch.autograd.Function):
    """
    we can implement our own autograd functions 
    by subclassing torch.autograd.Function and 
    implementing forward and backward passes.
    """

    @staticmethod
    def forward(context, input):
        context.save_for_backward(input)
        return input.clamp(min=0)

    @staticmethod
    def backward(context, grad_output):
        """
        in the backward pass, we recieve a input(torch.Tensor) containing the gradient of the loss
        with respect to the output, and we need to compute the gradient of the loss with respect to the input
        """
        input, = context.saved_tensors
        grad_input = grad_output.clone()
        grad_input[input < 0] = 0
        return grad_input


dtype = torch.float
device = torch.device("cpu")       # gpu = cuda

# N is batch size, Din is input dimension, 
# H is hidden layer dimension and Dout is output dimension
N, Din, H, Dout = 64, 1000, 100, 10

x = torch.randn(N, Din, device=device, dtype=dtype)
y = torch.randn(N, Dout, device=device, dtype=dtype)

w1 = torch.randn(Din, H, device=device, dtype=dtype, requires_grad=True)
w2 = torch.randn(H, Dout, device=device, dtype=dtype, requires_grad=True)

lr = 1e-6 
for e in range(500):
    relu = MyReLU.apply
    y_pred = relu(x.mm(w1)).mm(w2)

    loss = (y_pred - y).pow(2).sum()
    if e % 100 == 0:
        print(e, loss.item())

    loss.backward()

    with torch.no_grad():
        w1 -= lr * w1.grad
        w2 -= lr * w2.grad

        w1.grad.zero_()
        w2.grad.zero_()

# nn module
## PyTorch: nn

Computational graphs and autograd are a very powerful paradigm for defining complex operators and automatically taking derivatives; however for large neural networks raw autograd can be a bit too low-level.

When building neural networks we frequently think of arranging the computation into layers, some of which have learnable parameters which will be optimized during learning.

In TensorFlow, packages like Keras, TensorFlow-Slim, and TFLearn provide higher-level abstractions over raw computational graphs that are useful for building neural networks.

In PyTorch, the nn package serves this same purpose. The nn package defines a set of Modules, which are roughly equivalent to neural network layers. A Module receives input Tensors and computes output Tensors, but may also hold internal state such as Tensors containing learnable parameters. The nn package also defines a set of useful loss functions that are commonly used when training neural networks.

In this example we use the nn package to implement our two-layer network:

In [ ]:
import torch
from torch.nn import *

# N is batch size, Din is input dimension, 
# H is hidden layer dimension and Dout is output dimension
N, Din, H, Dout = 64, 1000, 100, 10

x = torch.randn(N, Din, device=device, dtype=dtype)
y = torch.randn(N, Dout, device=device, dtype=dtype)

criterion = MSELoss(reduction='sum')

# Sequential is a module which contains other modules, and applies them in sequence
# to produce its output. Each Linear Module is a Fully Connected Layer and holds 
# internal Tensor for its weights and bias.
model = Sequential(
    Linear(Din, H),
    ReLU(),
    Linear(H,Dout)
)

lr = 1e-6 
for e in range(500):
    y_pred = model(x)
    loss = criterion(y_pred, y)
    if e % 100 == 0:
        print(e, loss.item())
    
    model.zero_grad()
    loss.backward()

    with torch.no_grad():
        for param in model.parameters():
            param -= lr * param.grad

## PyTorch: optim

Up to this point we have updated the weights of our models by manually mutating the Tensors holding learnable parameters (with torch.no_grad() or .data to avoid tracking history in autograd). This is not a huge burden for simple optimization algorithms like stochastic gradient descent, but in practice we often train neural networks using more sophisticated optimizers like AdaGrad, RMSProp, Adam, etc.

The optim package in PyTorch abstracts the idea of an optimization algorithm and provides implementations of commonly used optimization algorithms.

In this example we will use the nn package to define our model as before, but we will optimize the model using the Adam algorithm provided by the optim package:

In [17]:
import torch
from torch.optim import Adam
from torch.nn import *

# N is batch size, Din is input dimension, 
# H is hidden layer dimension and Dout is output dimension
N, Din, H, Dout = 64, 1000, 100, 10

x = torch.randn(N, Din, device=device, dtype=dtype)
y = torch.randn(N, Dout, device=device, dtype=dtype)

lr = 1e-6
criterion = MSELoss(reduction='sum')
optimizer = Adam(model.parameters(), lr=lr)

for e in range(500):
    # Forward pass 
    y_pred = model(x)
    loss = criterion(y_pred, y)

    if e % 100 == 0:
        print(e, loss.item())

    optimizer.zero_grad()
    loss.backward()

    optimizer.step()

0 700.7799682617188
100 683.9097290039062
200 667.51123046875
300 651.6246337890625
400 636.155029296875


## PyTorch: Custom nn Modules

Sometimes you will want to specify models that are more complex than a sequence of existing Modules; for these cases you can define your own Modules by subclassing nn.Module and defining a forward which receives input Tensors and produces output Tensors using other modules or other autograd operations on Tensors.

In this example we implement our two-layer network as a custom Module subclass:

In [19]:
import torch
from torch.nn import *
from torch.optim import SGD

class TwoLayerNet(torch.nn.Module):
    def __init__ (self, Din, H, Dout):
        super(TwoLayerNet, self).__init__()
        self.linear1 = Linear(Din, H)
        self.linear2 = Linear(H, Dout)
    
    def forward(self, x):
        h_relu = self.linear1(x).clamp(min=0)
        return self.linear2(h_relu)
    
# N is batch size, Din is input dimension, 
# H is hidden layer dimension and Dout is output dimension
N, Din, H, Dout = 64, 1000, 100, 10

x = torch.randn(N, Din, device=device, dtype=dtype)
y = torch.randn(N, Dout, device=device, dtype=dtype)

lr = 1e-6
criterion = MSELoss(reduction='sum')
optimizer = SGD(model.parameters(), lr=lr)

model = TwoLayerNet(Din, H, Dout)

for e in range(500):
    y_pred = model(x)
    loss = criterion(y_pred, y)
    if e % 100 == 0:
        print(e, loss.item())
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


        

0 639.8817138671875
100 639.8817138671875
200 639.8817138671875
300 639.8817138671875
400 639.8817138671875


## PyTorch: Control Flow + Weight Sharing

As an example of dynamic graphs and weight sharing, we implement a very strange model: a fully-connected ReLU network that on each forward pass chooses a random number between 1 and 4 and uses that many hidden layers, reusing the same weights multiple times to compute the innermost hidden layers.

For this model we can use normal Python flow control to implement the loop, and we can implement weight sharing among the innermost layers by simply reusing the same Module multiple times when defining the forward pass.

We can easily implement this model as a Module subclass:

In [20]:
import random
import torch 

from torch.nn import *

class DynamicNet(torch.nn.Module):
    def __init__(self, Din, H, Dout):
        super(DynamicNet, self).__init__()
        self.input_linear = Linear(Din, H)
        self.middle_linear = Linear(H, H)
        self.output_linear = Linear(H, Dout)

    def forward (self, x):
        h_relu = self.input_linear(x).clamp(min=0)
        for _ in range(random.randint(0,3)):
             h_relu = self.middle_linear(h_relu).clamp(min=0)
        return self.output_linear(h_relu)

# N is batch size, Din is input dimension, 
# H is hidden layer dimension and Dout is output dimension
N, Din, H, Dout = 64, 1000, 100, 10

x = torch.randn(N, Din, device=device, dtype=dtype)
y = torch.randn(N, Dout, device=device, dtype=dtype)

lr = 1e-6
criterion = MSELoss(reduction='sum')
optimizer = SGD(model.parameters(), lr=lr)

model = DynamicNet(Din, H, Dout)

for e in range(500):
    y_pred = model(x)
    loss = criterion(y_pred, y)
    if e % 100 == 0:
        print(e, loss.item())
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

0 615.9288330078125
100 663.4950561523438
200 663.4950561523438
300 624.8159790039062
400 615.9288330078125
